<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/%D0%9A%D0%B0%D0%B7%D0%B0%D0%BD%D1%86%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import re
import pandas as pd

# Словарь предметов с вариантами названий
SUBJECTS = {
    "Математика": ["Математика", "Алгебра", "Геометрия"],
    "Русский язык": ["Русский язык"],
    "Литература": ["Литература"],
    "История": ["История"],
    "Обществознание": ["Обществознание"],
    "География": ["География"],
    "Биология": ["Биология", "Ботаника"],
    "Химия": ["Химия"],
    "Физика": ["Физика"],
    "Информатика": ["Информатика", "Кибербезопасность"],
    "Иностранный язык": ["Английский язык", "Немецкий язык", "Французский язык", "Испанский язык"],
    "Музыка": ["Музыка"],
    "Изобразительное искусство": ["Изобразительное искусство", "ИЗО"],
    "Технология": ["Технология"],
    "ОБЖ": ["Основы безопасности жизнедеятельности", "ОБЖ"],
    "Физкультура": ["Физическая культура", "Физкультура"],
    "ОДНКР": ["ОДНКР", "ОДНКНР"],
    "Естествознание": ["Естествознание", "Окружающий мир", "Экология"],
}

BASE_URL = "https://rulex.kpfu.ru/booklist"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Функция для получения списка ссылок на "Список терминов"
def get_term_links():
    response = requests.get(BASE_URL, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    links = [
        "https://rulex.kpfu.ru" + a["href"]
        for a in soup.find_all("a", string="Список терминов")
    ]
    return links

# Функция для извлечения предмета из названия учебника
def extract_subject(title):
    for subject, variations in SUBJECTS.items():
        if any(variant in title for variant in variations):
            return subject
    return title  # Если предмет не найден, возвращаем полный заголовок

# Функция для извлечения минимального класса из заголовка
def extract_min_class(title):
    matches = re.findall(r"(\d+)\s*[-.]?\s*(?:й|го|гo)?\s*(?:кл[.]?|класс)", title, re.IGNORECASE)
    roman_matches = re.findall(r"\b([IVXLCDM]+)\b\s*[-.]?\s*(?:кл[.]?|класс)", title, re.IGNORECASE)
    classes = [int(match) for match in matches]
    roman_to_int = {"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "VI": 6, "VII": 7, "VIII": 8, "IX": 9, "X": 10}
    for match in roman_matches:
        if match in roman_to_int:
            classes.append(roman_to_int[match])
    return min(classes) if classes else None

# Функция для парсинга одной страницы
def parse_page(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    class_info = soup.find("h4").text.strip() if soup.find("h4") else ""
    subject = extract_subject(class_info)
    min_class = extract_min_class(class_info)
    table = soup.find("table", class_="w3-table-all")
    terms = []
    if table:
        for row in table.find_all("tr")[1:]:
            cols = row.find_all("td")
            if len(cols) >= 2:
                term = cols[0].text.strip()
                frequency = int(cols[1].text.strip()) if cols[1].text.strip().isdigit() else 0
                terms.append((term, subject, min_class, frequency, class_info))  # Добавили class_info
    return terms, class_info

# Основная функция
def main():
    term_links = get_term_links()
    all_terms = []
    problematic_books = set()
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(parse_page, term_links)
    for result, book_title in results:
        all_terms.extend(result)
        if any(term[2] is None for term in result):
            problematic_books.add(book_title)
    print("Учебники, у которых не удалось определить минимальный класс:")
    for book in problematic_books:
        print(book)
    term_dict = {}
    for term, subject, min_class, frequency, book in all_terms:
        if term not in term_dict:
            term_dict[term] = {"subjects": {}, "frequency": 0}
        if subject not in term_dict[term]["subjects"]:
            term_dict[term]["subjects"][subject] = min_class
        else:
            term_dict[term]["subjects"][subject] = min(term_dict[term]["subjects"][subject], min_class)
        term_dict[term]["frequency"] += frequency
    df = pd.DataFrame([
        (
            term,
            ", ".join(sorted(term_data["subjects"].keys())),
            ", ".join(str(term_data["subjects"][subject]) for subject in sorted(term_data["subjects"])),
            term_data["frequency"]
        )
        for term, term_data in term_dict.items()
    ], columns=["Слово или сочетание", "Предмет", "Минимальный класс, где встречается", "Частота"])
    df.to_excel("terms.xlsx", index=False)
    print("Файл terms.xlsx успешно создан!")
    search_term = "велит"

    # Ищем учебники, в которых встречается нужное слово
    books_with_term = set()
    for term, subject, min_class, frequency, book in all_terms:
        if term == search_term:
           books_with_term.add(book)

    # Выводим результат
    print(f"Слово '{search_term}' встречается в следующих учебниках:")
    for book in books_with_term:
        print(book)

if __name__ == "__main__":
    main()

Учебники, у которых не удалось определить минимальный класс:
Файл terms.xlsx успешно создан!
Слово 'велит' встречается в следующих учебниках:
История: Юдовская А. Я. Всеобщая история. История Нового времени, 1500-1800. 7 класс: учеб . для общеобразоват. организаций / А. Я. Юдовская, П. А. Баранов, Л. М. Ванюшкина ; под ред. А. А. Искендерова. - 6-е изд. - М. : Просвещение, 2018. - 319 с., [16] л. : ил., карт. - ISBN 978-5-09-055148-9.
История: Черникова, Татьяна Васильевна. История России с древнейших времён до начала XVI века. 6 класс: учебник/ Т. В. Черникова, К. П. Чиликин ; под общ. ред. В. Р. Мединского. – Москва : Просвещение, 2021. - 272 с. : ил., карты. ISBN 978-5-09-081267-2.
История: "Никишин В. О. История. Всеобщая история. История Древнего мира: учебник для 5 класса общеобразовательных организаций / В. О. Никишин, А. В. Стрелков, О. В. Томашевич, Ф.А. Михайловский; под науч. ред. С. П. Карпова. — М.: ООО «Русское слово — учебник», 2023. — 328 с.: ил. — (ФГОС 2021. Инновацио